In [ ]:
import sys
sys.path.insert(0, '..')

In [ ]:
from python_environment_check import check_packages

d = {
    'numpy': '1.21.2',
    'matplotlib': '3.4.3',
    'pandas': '1.3.2'
}
check_packages(d)

An Object-oriented perceptron API

In [ ]:
import numpy as np

np.where( 1.2 >= 0.0, 1, 0)

In [ ]:
import numpy as np

class Perceptron:
    """Perceptron classifier.

    Parameters
    ----------
    eta : float
        Learning rate(between 0.0 and 1.0)
    n_iter : int
        Passes over the training dataset.
    random_state : int
        Random number generator seed for random weight initialization. 

    Attributes
    ----------
    w_ : 1d-array
        Weights after fitting. 
    b_ : scalar
        Bias unit after fitting.
    errors_ : list
        Number of misclassifications (updates) in each epoch.
    """
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state
    
    def fit(self, X, y):
        """Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_examples, n_features]
            Training vectors, where n_examples is the number of examples and n_features is the number of features. 
        y : array-like, shape = [n_examples]
            Target values.
        
        Returns
        -----------
        self : object
        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=X.shape[1])
        self.b_ = np.float_(0.)

        self.errors_ = []

        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi)) # learning rate * (true class - predicted class)
                self.w_ += update * xi # add update proportional to size of feature xj for ith training example
                self.b_ += update # update bias
                errors += int(update != 0.0) # count if prediction was wrong
            self.errors_.append(errors)
        return self

    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_) + self.b_ # calculate z *predicted class)
        
    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, 0) # when z (dot protduct) is greater than 0, yield 1 class label, otherwise yield 0



# Training a perceptron model on the iris dataset

In [ ]:
import os 
import pandas as pd

try: 
    s = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
    print('From URL:', s)
    df = pd.read_csv(s,
                     header=None,
                     encoding='utf-8')

except HTTPError:
    s = 'iris.data'
    print('From local Iris path:', s)
    df = pd.read_csv(s,
                     header=None,
                     encoding='utf-8')

df. tail()

### Plotting the Iris data

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# select setosa and versicolor
y = df.iloc[0:100, 4].values
y = np.where(y == 'Iris-setosa', 0, 1)

# extract sepal length and petal length
X = df.iloc[0:100, [0,2]].values

# plot data
plt.scatter(X[:50, 0], X[:50,1],
            color='red', marker='o', label='Setoasa')
plt.scatter(X[50:100, 0], X[50:100, 1],
            color='blue', marker='s', label='Versicolor')

plt.xlabel('Sepal length [cm]')
plt.ylabel('Petal length [cm]')
plt.legend(loc='upper left')

plt.show()

### Training the perceptron model

In [ ]:
ppn = Perceptron(eta=0.1, n_iter=10)

ppn.fit(X,y)

plt.plot(range(1, len(ppn.errors_)+1), ppn.errors_, marker='o')
plt.xlabel('Epochs')
plt.ylabel('Number of updates')

plt.show()

In [ ]:
x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, 0.02),
                            np.arange(x2_min, x2_max, 0.02))

xx1, xx2
np.shape(xx1)
np.shape(xx1.ravel())

lab = ppn.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
np.shape(lab)
print(np.shape(lab))
lab = lab.reshape(xx1.shape)
print(np.shape(lab))

In [ ]:
from matplotlib.colors import ListedColormap

def plot_decision_regions(X, y, classifier, resolution=0.02):

    # setup marker generator and color map
    markers = ('o', 's', '^', 'v', '<')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1 # set limits of grid to try (add padding of 1 unit at the edges)
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                            np.arange(x2_min, x2_max, resolution)) # make a grid of xx1 and xx2 to try and predict
    lab = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T) # unravel the xx1 to look like x1 and x2 features
    lab = lab.reshape(xx1.shape) # reshape it back into the meshgrid shape for plotting
    plt.contourf(xx1, xx2, lab, alpha = 0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # plot class examples
    for idx, cl in enumerate(np.unique(y)): # enumerate allows you to access the index as well as the values of y.. cool!
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.8, 
                    c=colors[idx],
                    marker=markers[idx], 
                    label=f'Class {cl}', 
                    edgecolor='black')


In [ ]:
plot_decision_regions(X, y, classifier=ppn)
plt.xlabel('Sepal length [cm]')
plt.ylabel('Petal length [cm]')
plt.legend(loc='upper left')


#plt.savefig('images/02_08.png', dpi=300)
plt.show()

# Implementing an adaptive linear neuron in Python

In [ ]:
class AdalineGD:
    """ Adaptive linear neuron classifier. 
    
    Parameters
    ------
    eta : float
        Learning rate (between 0.0 and 1.0)
    n_iter : int
        Passes over the training dataset. 
    random_state : int
        Random number generator seed for random weight intialization. 

    Attributes
    ----------
    w_ : 1d-array
        weights after fitting. 
    b_ : Scalar
        Bias unit after fitting. 
    losses_ : list
        Mean squared error loss function values in each epoch
    """

    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    def fit(self, X, y):
        """Fit training data.

        Parameters
        ---------
        X : {array-like}, shape = [n_examples, n_features]
            Training vectors, where n_examples is the number of examples and n_features is the number of features. 
        y : array-like, shape = [n_examples]
            Target values. 
        
        Returns
        -------
        self : object

        """

        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size = X.shape[1])
        self.b_ = np.float(0.)
        self.lossews = []

        for i in range(self.n_iter):
            net_input = self.net_input(X)
            # Please note that the "activation" method has no effect 
            # in the code since it is simply an identity function. We 
            # could write 'output = self.net_input(X)' directly isntead.
            # The purpose of the activation is more conceptual, i.e., 
            # in the case of logistic regression (as we will see later), 
            # we could change it to 
            # a sigmoid function to implement a logistic regression classifier
            output = self. activation(net_input) # shape should be equal to y -> [n_examples]
            errors = (y - output) 

            self.w_ += self.eta * 2.0 * X.T.dot(errors) / X.shape[0] # [n_examples, n_features].T * [n_examples]
            self.b_ += self.eta * 2.0 * errors.mean()
            loss = (errors**2).mean()
            self.losses_.append(loss)
        return self

    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_) + self.b_ # shape should be equal to y [n_examples]
    
    def activation(self, X):
        """Compute linear activation"""
        return X 
    
    def predict(self, X):
        """ Return class label after unit step"""
        return np.where(self.activation(self.net_input(X)) >= 0.5, 1, 0)


In [ ]:
# Matrix multiplication testing 

foo = np.array([[1,2,3], [4,5,6]])
print(foo.shape)
faa = np.array([[7],[8]])
print(faa.shape)
foo.T.dot(faa)
